## Data validation and settings management using Pydantic. Type annotations.

In [ ]:
%pip install pydantic

### Basic use case

In [7]:
from pydantic import BaseModel, ConfigDict

class User(BaseModel):
    model_config = ConfigDict(strict=True)
    name: str
    age: int

#user = User(name="Alice", age="30")
# pydantic.error_wrappers.ValidationError
# age

user = User(name="Alice", age=30)
print(user)

name='Alice' age=30


### Beyond "str" and "int"
Pydantic supports below field types
1. Primitive types: str, int, float, bool
2. Collection types: list, tuple, set, dict
3. Optional types: Optional from the typing module for fields that can be None
4. Fields are required by default unless explicitly marked as Optional.
5. Missing required fields will raise ValidationError.

In [11]:
from typing import List, Dict, Optional, Union
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: Optional[int]
    tags: List[str]
    metadata: Dict[str, Union[str, int, float]]

user1 = User(
    name="Alice",
    age=30,
    tags=["developer", "python"],
    metadata={"location": "Wonderland", "experience": 5}
)

### Field Constraints and Default Values 
This allows you to define validation rules and fallback values directly in your Pydantic models, ensuring data quality while providing flexibility.

Use Field() to add constraints like min/max values, string lengths, and regex patters
Specify defaults for optional fields with default= or use default_factory=for mutable defaults.

default_factory is used when you need to provide a dynamic value for a field, especially useful for mutable types like lists, dictionaries, or custom objects. Unlike default, which uses a **fixed value**, default_factory calls a function to **generate the default value** each time it's needed.

In [14]:
from pydantic import BaseModel, Field
from typing import List

class Product(BaseModel):
    id : int = Field(gt=0, description="Product ID must be positive")
    name: str = Field(min_length=3, max_length=50)
    price: float = Field(gt=0, lt=10000, default=9.99)
    in_stock: bool = Field(default=True)
    
# instantiate
product = Product(id=101, name="Widget")

In [ ]:
# difference between default and defualt_factory
# Notice empty list

from pydantic import BaseModel, Field
from typing import List
from datetime import datetime

class Order(BaseModel):
    # Bad: All instances will share the same list
    items_bad: List[str] = Field(default=[])

    # Good: Each instance gets a new list
    items_good: List[str] = Field(default_factory=list)

    # Generate timestamp when order is created
    created_at: datetime = Field(default_factory=datetime.now)

order1 = Order()
order2 = Order()
print("order1 = \n", order1, "\n" ,"order2 = \n", order2)

order1 = 
 items_bad=[] items_good=[] created_at=datetime.datetime(2025, 5, 27, 13, 41, 3, 678160) 
 order2 = 
 items_bad=[] items_good=[] created_at=datetime.datetime(2025, 5, 27, 13, 41, 3, 678223)


#### Pass the function reference without calling it
default_factory=list
default_factory=list()
The factory function is called only when a value is needed, making it efficient for resource-intensive operations.

In [29]:
# With custom factory functions

from pydantic import BaseModel, Field
import random
from datetime import datetime, timedelta

def generate_order_id():
    return f"ORD-{random.randint(1000, 9999)}"


class New_Order(BaseModel):
    order_id: str = Field(default_factory=generate_order_id)
    items: List[str] = Field(default_factory=list)
    expiry_date: datetime = Field(default_factory=lambda: datetime.now() + timedelta(days=30))

order1 = New_Order()
print(order1)

order_id='ORD-3281' items=[] expiry_date=datetime.datetime(2025, 6, 26, 13, 52, 38, 262288)


### Nested Models
Nest models within each other, enabling complex data structures.  
When defining nested models, Pydantic handles validation of the entire object tree, ensuring that data at all levels meets your specified requirements.

In [32]:
from pydantic import BaseModel
from typing import List

class Address(BaseModel):
    city: str
    country: str

# Parent model with nested Address
class Person(BaseModel):
    name: str
    addresses: List[Address]

# Example usage
person = Person(
    name="Alice",
    addresses=[
        Address(city="Wonderland", country="Fantasyland"),
        Address(city="Springfield", country="USA")
    ]   
)

### Custom validators

Custom validators enable complex validation logic beyond simple type checking, allowing for data transformation, cross-field validation, and business rule enforcement

* Validators can both validate and tranform input data
* Validation errors provide specific feedback about what went wrong
* Validators are executed in a predictable order during model creation

In [39]:
from pydantic import BaseModel, field_validator

class Product(BaseModel):
    name: str
    price: float

    @field_validator('price')
    def price_must_be_positive(cls, value):
        if value < 0:
            raise ValueError('Price should be positive')
        return value * 0.9 # data tranformation
    
# create a product with validation
product = Product(name="Mug", price=1)

print(product)

name='Mug' price=0.9
